In [101]:
import pandas as pd
import numpy as np
import os
#os.makedirs('work', exist_ok=True)
#Employee info
employee_info_df= pd.read_csv('Employee.csv')
employee_travel_df= pd.read_csv('EmployeeTravel.csv')
employee_vehicle_distance_df = pd.read_csv('EmplyeeVehicleDistance.csv')
motorbike_emission_df = pd.read_csv('motorBikeEmission.csv')
car_emission_df = pd.read_csv('carEmission.csv')
bus_taxi_emission_df = pd.read_csv('BusTaxiEmission.csv')
#Carbon emission lookup for rail, subway, tram
rail_subway_tram_emisison_df = pd.read_csv('Rail.csv')
#Telework
tele_emp_df = pd.read_csv("telework.csv")
tele_elec_df = pd.read_csv("electricityFactor.csv")
home_appliances_df= pd.read_csv("homeAppliance.csv")
home_consumption_df = pd.read_csv("homeConsumption.csv")
ave_heating_emission_df = pd.read_csv("heatEmissionTelework.csv")
avg_fuel_price=pd.read_csv("FuelPrice.csv")
fuel_emission_per_litre= pd.read_csv("FuelEmissionPerLitre.csv")

tele_elec_df.rename(columns = {'Country':'Work Region/Country'}, inplace = True)
ave_home_office_setup_consumption= 0.90
ave_heating_emission_factor= ave_heating_emission_df['Heat Emission Factor'].mean()

#Added newly
employee_travel_df.fillna(0, inplace=True)
employee_travel_df['Total Office Weeks / year'] =(employee_travel_df['Total Weeks Travel/ Year']-employee_travel_df['Telework Weeks / Year'])
employee_travel_df 

colsSet= {
"Distance between home and Office in Km": 0,
"Total of days travelled/week": 0,
"Total Weeks Travel/ Year": 0,
"Telework Weeks / Year":1,
"Emissions Factor (kgCO2pkm-1)":0,
"No of Emp in Group":1,
"Emission Factor (kgCO2e/L)":0
}

output_col = ['Employee Name', 'Employee Id', 'Organization Name', 'Work Region/Country', 'Transport Mode', 'Fuel Type', 'CO2e_Calculation_Method', 'Scope' ,'Year','Carbon Emmission/emp(kgCO2)']

# Have combined data for employee and travel details
employee_travel_info_df= pd.merge(employee_info_df, employee_travel_df, on=['Employee Name', 'Employee Id', 'Organization Name','Work Region/Country'], how="outer")

employee_travel_vehicle_df = pd.merge(employee_travel_info_df, employee_vehicle_distance_df, on=['Employee Name', 'Employee Id', 'Organization Name','Work Region/Country'], how="outer")
#######################################################################################################
# Fuel Based

emp_df = pd.merge(employee_travel_vehicle_df, fuel_emission_per_litre, on=['Fuel Type'], how="left" )
emp_final_df=pd.merge(emp_df,tele_elec_df, on=['Work Region/Country'], how="inner")

emp_join_car_emi_df=pd.merge(emp_final_df,car_emission_df, on=['Vehicle Category', 'Vehicle Sub-Category', 'Fuel Type', 'Year', 'Work Region/Country'], how='inner' )
emp_join_car_emi_df.fillna(colsSet, inplace=True)
emp_join_car_emi_df = emp_join_car_emi_df.drop_duplicates()

emp_join_bike_emi_df = pd.merge(emp_final_df, motorbike_emission_df, on=['Vehicle Category','Vehicle Sub-Category', 'Fuel Type','Work Region/Country','Year'], how='inner' )
emp_join_bike_emi_df.fillna(colsSet, inplace=True)
emp_join_bike_emi_df = emp_join_bike_emi_df.drop_duplicates()

emp_join_bus_taxi_df = pd.merge(emp_final_df,bus_taxi_emission_df, on=['Vehicle Category', 'Vehicle Sub-Category', 'Fuel Type','Work Region/Country','Year'], how='inner')
emp_join_bus_taxi_df.fillna(colsSet, inplace=True)
emp_join_bus_taxi_df = emp_join_bus_taxi_df.drop_duplicates()

rail_subway_tram_emisison_df.rename(columns = {'Country / Region':'Work Region/Country'}, inplace = True)
emp_join_rail_df = pd.merge(emp_final_df, rail_subway_tram_emisison_df, on=['Work Region/Country', 'Vehicle Category', 'Fuel Type','Vehicle Sub-Category', 'Year'], how='inner' )
emp_join_rail_df.fillna(colsSet, inplace=True)
emp_join_rail_df = emp_join_rail_df.drop_duplicates()

#Fuel based

emp_join_car_emi_df['Carbon Fuel Emmission/emp(kgCO2)'] = emp_join_car_emi_df.apply(lambda x: x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x["Quantity Of Electricity Consumed(Kwh/Km)"]*x["Emission Factor (kgCO2e/kWh)"]
                                                                         if x['Fuel Type']=="Battery Electric Vehicle"
                                                                         else x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x['Fuel Efficiency(L/Km)']*x['Emission Factor (kgCO2e/L)'], axis=1)

emp_join_bus_taxi_df['Carbon Fuel Emmission/emp(kgCO2)'] = emp_join_bus_taxi_df.apply(lambda x: x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x["Quantity Of Electricity Consumed(Kwh/Km)"]*x["Emission Factor (kgCO2e/kWh)"]
                                                                         if x['Fuel Type']=="Battery Electric Vehicle"
                                                                         else x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x['Fuel Efficiency(L/Km)']*x['Emission Factor (kgCO2e/L)'], axis=1)

emp_join_bike_emi_df['Carbon Fuel Emmission/emp(kgCO2)'] = emp_join_bike_emi_df.apply(lambda x: x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x["Quantity Of Electricity Consumed(Kwh/Km)"]*x["Emission Factor (kgCO2e/kWh)"]
                                                                         if x['Fuel Type']=="Battery Electric Vehicle"
                                                                         else x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x['Fuel Efficiency(L/Km)']*x['Emission Factor (kgCO2e/L)'], axis=1)

emp_join_rail_df['Carbon Fuel Emmission/emp(kgCO2)'] = emp_join_rail_df.apply(lambda x: x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x["Quantity Of Electricity Consumed(Kwh/Km)"]*x["Emission Factor (kgCO2e/kWh)"]
                                                                         if x['Fuel Type']=="Battery Electric Vehicle"
                                                                         else x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x['Fuel Efficiency(L/Km)']*x['Emission Factor (kgCO2e/L)'], axis=1)



emp_join_bus_taxi_df['Transport Mode'] = emp_join_bus_taxi_df['Vehicle Category']
emp_join_bus_taxi_df['CO2e_Calculation_Method_Fuel'] = "Fuel Based"
emp_join_bus_taxi_df['Scope'] = "Scope 3"
#final_emp_bus_taxi_emi = emp_join_bus_taxi_df[output_col]

emp_join_bike_emi_df['Transport Mode'] = emp_join_bike_emi_df['Vehicle Category']
emp_join_bike_emi_df['CO2e_Calculation_Method_Fuel'] = "Fuel Based"
emp_join_bike_emi_df['Scope'] = "Scope 3"
#final_emp_bike_emi = emp_join_bike_emi_df[output_col]

emp_join_car_emi_df['Transport Mode'] = emp_join_car_emi_df['Vehicle Category']
emp_join_car_emi_df['CO2e_Calculation_Method_Fuel'] = "Fuel Based"
emp_join_car_emi_df['Scope'] = "Scope 3"
#final_emp_car_emi = emp_join_car_emi_df[output_col]

emp_join_rail_df['Transport Mode'] = emp_join_rail_df['Vehicle Category']
emp_join_rail_df['CO2e_Calculation_Method_Fuel'] = "Fuel Based"
emp_join_rail_df['Scope'] = "Scope 3"
#final_join_rail_df = emp_join_rail_df[output_col]

output_col_final = ['Employee Name', 'Employee Id', 'Organization Name', 'Work Region/Country', 'Transport Mode', 'Fuel Type', 'CO2e_Calculation_Method', 'Scope' ,'Year','Carbon cooling Emmission/emp(kgCO2)','Carbon home appliances Emmission/emp(kgCO2)','Carbon heating Emmission/emp(kgCO2)','Carbon Emmission/emp(kgCO2)']



#Concatenating the sub data frames


final_df_bus_taxi_bike = pd.concat([emp_join_bus_taxi_df, emp_join_bike_emi_df], axis=0, ignore_index=True)
final_df_car_rail = pd.concat([emp_join_car_emi_df, emp_join_rail_df], axis=0, ignore_index=True)
final_df_all_vehicle_fuel =pd.concat([final_df_bus_taxi_bike,final_df_car_rail], axis=0, ignore_index=True)
final_df_all_vehicle_fuel['Carbon cooling Emmission/emp(kgCO2)'] = 0
final_df_all_vehicle_fuel['Carbon home appliances Emmission/emp(kgCO2)'] = 0
final_df_all_vehicle_fuel['Carbon heating Emmission/emp(kgCO2)'] = 0
#final_df_all_vehicle_fuel = final_df_all_vehicle[output_col_final]
final_df_all_vehicle_fuel.fillna({"Carbon Fuel Emmission/emp(kgCO2)":0}, inplace=True)

###################################################################################################

#Handling electric emission



emp_join_car_emi_df['Carbon Car Emmission/emp(kgCO2)'] = emp_join_car_emi_df.apply(lambda x: x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x["Electricity Emissions Factor (kgCO2vkm-1)"]
                                                                                     if x['Fuel Type']=="Battery Electric Vehicle"
                                                                                     else x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x['Emissions Factor (kgCO2vkm-1)'], axis=1)

emp_join_bus_taxi_df['Carbon Bus Taxi Emmission/emp(kgCO2)'] = emp_join_bus_taxi_df.apply(lambda x: x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x["Electricity Emissions Factor (kgCO2vkm-1)"]
                                                                                     if x['Fuel Type']=="Battery Electric Vehicle"
                                                                                     else x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x['Emissions Factor (kgCO2pkm-1)'], axis=1)

emp_join_bike_emi_df['Carbon Bike Emmission/emp(kgCO2)'] = emp_join_bike_emi_df.apply(lambda x: x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x["Electricity Emissions Factor (kgCO2vkm-1)"]
                                                                                     if x['Fuel Type']=="Battery Electric Vehicle"
                                                                                     else x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x['Emissions Factor (kgCO2vkm-1)'], axis=1)

emp_join_rail_df['Carbon Rail Emmission/emp(kgCO2)'] = emp_join_rail_df.apply(lambda x: x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x["Electricity Emissions Factor (kgCO2vkm-1)"]
                                                                                     if x['Fuel Type']=="Battery Electric Vehicle"
                                                                                     else x['Distance between home and Office in Km']*2*x['Total of days travelled/week']*x['Total Office Weeks / year']*x['Emissions Factor (kgCO2pkm-1)'], axis=1)



# Handling Pool 
emp_join_car_emi_df['Carbon Emmission Pool/emp(kgCO2)'] = emp_join_car_emi_df.apply(lambda x: x['No of Days Travelled in Pool/Week']*x['No of Weeks Travelled in Pool/Year']*x['Distance between home and Office in Km']*2*(x['Emissions Factor (kgCO2vkm-1)']/x['No of Emp in Group'])
                                                                                    if x['Travelled with Pool (Y/N)?'] =="Y"
                                                                                    else 0, axis=1)

emp_join_bus_taxi_df['Carbon Emmission Pool/emp(kgCO2)'] = emp_join_bus_taxi_df.apply(lambda x:  x['No of Days Travelled in Pool/Week']*x['No of Weeks Travelled in Pool/Year']*x['Distance between home and Office in Km']*2*(x['Emissions Factor (kgCO2pkm-1)']/x['No of Emp in Group'])
                                                                                      if x['Travelled with Pool (Y/N)?'] =="Y"
                                                                                      else 0, axis=1)

emp_join_bike_emi_df['Carbon Emmission Pool/emp(kgCO2)'] = emp_join_bike_emi_df.apply(lambda x:  x['No of Days Travelled in Pool/Week']*x['No of Weeks Travelled in Pool/Year']*x['Distance between home and Office in Km']*2*(x['Emissions Factor (kgCO2vkm-1)']/x['No of Emp in Group'])
                                                                                      if x['Travelled with Pool (Y/N)?'] =="Y"
                                                                                      else 0, axis=1)

emp_join_rail_df['Carbon Emmission Pool/emp(kgCO2)'] = emp_join_rail_df.apply(lambda x: x['No of Days Travelled in Pool/Week']*x['No of Weeks Travelled in Pool/Year']*x['Distance between home and Office in Km']*2*(x['Emissions Factor (kgCO2pkm-1)']/x['No of Emp in Group'])
                                                                         if x['Travelled with Pool (Y/N)?'] =="Y"
                                                                         else 0, axis=1)



# Finalizing the emission
emp_join_car_emi_df['Carbon Distance Emmission/emp(kgCO2)'] = emp_join_car_emi_df['Carbon Car Emmission/emp(kgCO2)']-emp_join_car_emi_df['Carbon Emmission Pool/emp(kgCO2)']
emp_join_bus_taxi_df['Carbon Distance Emmission/emp(kgCO2)'] = emp_join_bus_taxi_df['Carbon Bus Taxi Emmission/emp(kgCO2)']-emp_join_bus_taxi_df['Carbon Emmission Pool/emp(kgCO2)']
emp_join_bike_emi_df['Carbon Distance Emmission/emp(kgCO2)'] = emp_join_bike_emi_df['Carbon Bike Emmission/emp(kgCO2)']-emp_join_bike_emi_df['Carbon Emmission Pool/emp(kgCO2)']
emp_join_rail_df['Carbon Distance Emmission/emp(kgCO2)'] = emp_join_rail_df['Carbon Rail Emmission/emp(kgCO2)']-emp_join_rail_df['Carbon Emmission Pool/emp(kgCO2)']


output_col_final = ['Employee Name', 'Employee Id', 'Organization Name', 'Work Region/Country', 'Transport Mode', 'Fuel Type', 'CO2e_Calculation_Method', 'Scope' ,'Year','Carbon cooling Emmission/emp(kgCO2)','Carbon home appliances Emmission/emp(kgCO2)','Carbon heating Emmission/emp(kgCO2)','Carbon Emmission/emp(kgCO2)']
                   
# Teleworking
"""Total home office setup:
Laptop/desktop, Computer monitor, Printer, Office light, Kettle
coumption = 0.90 kWh/day(8 hours)
for air conditioner = 2.45 Kw/hour 2, Assuming 4 hours per day and half of total telework days (summer and winter)
ave_heating_emission_factor(kgCo2/kWh)== 0.2304665
"""
# Tele_emp_df need to remove and want to add Info into employee_travel_df(Pending)
#emp_tele_travel_df = pd.merge(employee_travel_df, tele_elec_df, on=['Work Region/Country'], how='left')
emp_tele_travel_df = pd.merge(tele_emp_df, tele_elec_df, on=['Work Region/Country'], how='left')

emp_tele_mid_df =  pd.merge(emp_tele_travel_df, home_appliances_df, on=['Work Region/Country'], how='left')
emp_tele_full_df = pd.merge(emp_tele_mid_df, home_consumption_df, on=['Work Region/Country'], how='left')
emp_tele_full_df.fillna({"Telework Days/Week":0,"Telework Weeks":0, "Telework Weeks in winter":0, "Telework Weeks in Summer":0}, inplace=True)
emp_tele_full_df = emp_tele_full_df.drop_duplicates()

emp_tele_full_df['Carbon cooling Emmission/emp(kgCO2)'] = (2.45*4*emp_tele_full_df['Telework Days/Week']*emp_tele_full_df['Telework Weeks in winter / Year']*emp_tele_travel_df['Emission Factor (kgCO2e/kWh)'])
emp_tele_full_df['Carbon home appliances Emmission/emp(kgCO2)'] = (ave_home_office_setup_consumption*8*emp_tele_full_df['Telework Days/Week']*emp_tele_full_df['Telework Weeks / Year']*emp_tele_travel_df['Emission Factor (kgCO2e/kWh)'])
emp_tele_full_df['Carbon heating Emmission/emp(kgCO2)'] = (emp_tele_full_df['Telework Days/Week']*emp_tele_full_df['Telework Weeks in Summer / Year']*(emp_tele_full_df['kWh/home/year consumption']/365)/24*8*ave_heating_emission_factor)
emp_tele_travel_df = emp_tele_full_df.copy() 
emp_tele_travel_df['Carbon Emmission/emp(kgCO2)'] = (emp_tele_travel_df['Carbon cooling Emmission/emp(kgCO2)'] + emp_tele_travel_df['Carbon home appliances Emmission/emp(kgCO2)'] + emp_tele_travel_df['Carbon heating Emmission/emp(kgCO2)'])
###############################################################################################################################################################


emp_join_bus_taxi_df['Transport Mode'] = emp_join_bus_taxi_df['Vehicle Category']
emp_join_bus_taxi_df['CO2e_Calculation_Method_Distance'] = "Distance Based"
emp_join_bus_taxi_df['Scope'] = "Scope 3"


emp_join_bike_emi_df['Transport Mode'] = emp_join_bike_emi_df['Vehicle Category']
emp_join_bike_emi_df['CO2e_Calculation_Method_Distance'] = "Distance Based"
emp_join_bike_emi_df['Scope'] = "Scope 3"


emp_join_car_emi_df['Transport Mode'] = emp_join_car_emi_df['Vehicle Category']
emp_join_car_emi_df['CO2e_Calculation_Method_Distance'] = "Distance Based"
emp_join_car_emi_df['Scope'] = "Scope 3"


emp_join_rail_df['Transport Mode'] = emp_join_rail_df['Vehicle Category']
emp_join_rail_df['CO2e_Calculation_Method_Distance'] = "Distance Based"
emp_join_rail_df['Scope'] = "Scope 3"

#Concatenating the sub data frames

final_df_bus_taxi_bike = pd.concat([emp_join_bus_taxi_df, emp_join_bike_emi_df], axis=0, ignore_index=True)
final_df_car_rail = pd.concat([emp_join_car_emi_df, emp_join_rail_df], axis=0, ignore_index=True)
final_df_all_vehicle =pd.concat([final_df_bus_taxi_bike,final_df_car_rail], axis=0, ignore_index=True)

#Joining fuel and distance 

fuel_dist_df = pd.merge(final_df_all_vehicle_fuel, final_df_all_vehicle, on=['Employee Name', 'Employee Id', 'Organization Name','Work Region/Country','Year'], how="inner" )




fuel_dist_df['Carbon Emmission/emp(kgCO2)'] = fuel_dist_df.apply(lambda x : x['Carbon Distance Emmission/emp(kgCO2)']
                                                                         if (x['Carbon Fuel Emmission/emp(kgCO2)_x']==0)
                                                                         else x['Carbon Fuel Emmission/emp(kgCO2)_x'], axis=1)



fuel_dist_df['CO2e_Calculation_Method'] = fuel_dist_df.apply(lambda x : "Distance Method"
                                                                         if (x['Carbon Fuel Emmission/emp(kgCO2)_x']==0)
                                                                        else x['CO2e_Calculation_Method_Fuel_x'], axis=1)


fuel_dist_df['Carbon cooling Emmission/emp(kgCO2)'] = 0
fuel_dist_df['Carbon home appliances Emmission/emp(kgCO2)'] = 0
fuel_dist_df['Carbon heating Emmission/emp(kgCO2)'] = 0

fuel_dist_df['Transport Mode'] = fuel_dist_df['Vehicle Category_x']
fuel_dist_df['Scope'] = "Scope 3"
fuel_dist_df['Fuel Type'] = fuel_dist_df['Fuel Type_x']

fuel_dist_df = fuel_dist_df[output_col_final]

emp_tele_travel_df['Transport Mode'] = "Telework"
emp_tele_travel_df['CO2e_Calculation_Method'] = "Teleworking"
emp_tele_travel_df['Scope'] = "Scope 3"
emp_tele_travel_df['Fuel Type'] = "Not admissible"

#output columns and joining for the telework

final_join_tele_df = emp_tele_travel_df[output_col_final]

final_join_tele_df.fillna(0, inplace=True)

final_df =pd.concat([fuel_dist_df,final_join_tele_df], axis=0, ignore_index=True)
#final_df =pd.concat([final_df_all_vehicle,final_join_tele_df], axis=0, ignore_index=True)


final_df['Carbon Emmission/emp(kgCO2)'] = np.round(final_df['Carbon Emmission/emp(kgCO2)'], decimals = 3)
final_df['Carbon cooling Emmission/emp(kgCO2)'] = np.round(final_df['Carbon cooling Emmission/emp(kgCO2)'], decimals = 3)
final_df['Carbon home appliances Emmission/emp(kgCO2)'] = np.round(final_df['Carbon home appliances Emmission/emp(kgCO2)'], decimals = 3)
final_df['Carbon heating Emmission/emp(kgCO2)'] = np.round(final_df['Carbon heating Emmission/emp(kgCO2)'], decimals = 3)

final_df_output = final_df.groupby(by=['Employee Id', 'Transport Mode'], as_index=False).agg({'Carbon Emmission/emp(kgCO2)' :'sum'})
final_df_output['Employee Name'] = final_df['Employee Name']
final_df_output['Employee Id'] = final_df['Employee Id']
final_df_output['Organization Name'] = final_df['Organization Name']
final_df_output['Work Region/Country'] = final_df['Work Region/Country']
final_df_output['Transport Mode'] = final_df['Transport Mode']
final_df_output['Fuel Type'] = final_df['Fuel Type']
final_df_output['CO2e_Calculation_Method'] = final_df['CO2e_Calculation_Method']
final_df_output['Scope'] = final_df['Scope']
final_df_output['Year'] = final_df['Year']
final_df_output['Carbon cooling Emmission/emp(kgCO2)'] = final_df['Carbon cooling Emmission/emp(kgCO2)']
final_df_output['Carbon home appliances Emmission/emp(kgCO2)'] = final_df['Carbon home appliances Emmission/emp(kgCO2)']
final_df_output['Carbon heating Emmission/emp(kgCO2)'] = final_df['Carbon heating Emmission/emp(kgCO2)']
final_df_output['Carbon Emmission/emp(kgCO2)'] = final_df['Carbon Emmission/emp(kgCO2)']



# Handling Bycle and Foot 
final_df_output['Carbon Emmission/emp(kgCO2)'] = final_df_output.apply(lambda x : 0 if x['Transport Mode'] =="Foot" or x['Transport Mode']=="Bycle" else x['Carbon Emmission/emp(kgCO2)'], axis=1)

final_df_output.fillna({'Carbon Emmission/emp(kgCO2)':0,
                        'Carbon cooling Emmission/emp(kgCO2)':0,
                        'Carbon home appliances Emmission/emp(kgCO2)':0,
                        'Carbon heating Emmission/emp(kgCO2)':0                      
                       }, inplace =True)
final_df_output['Employee Id'] = final_df_output['Employee Id'].astype(int)
final_df_output['Year'] = final_df_output['Year'].astype(int)
final_df_output
#Ave Data Method 

total_emp_count = employee_info_df['Employee Id'].unique().size
total_emp_by_each_mode =  employee_travel_vehicle_df.groupby(by='Vehicle Category', as_index=False).count()
total_emp_by_each_mode['Total emp by each mode'] = total_emp_by_each_mode['Employee Id']
total_emp_by_each_mode['Emp % of each mode'] = (total_emp_by_each_mode['Employee Id']/total_emp_count)*100



avg_distance_each_mode= employee_travel_vehicle_df.groupby(by=['Vehicle Category'], as_index=False).agg({'Distance between home and Office in Km':'mean'})
avg_distance_each_mode.rename(columns = {'Distance between home and Office in Km': 'Avg distance of each mode'}, inplace = True)


avg_emp_join_car_emi_df = emp_join_car_emi_df.groupby(by=['Vehicle Category'], as_index=False).agg({'Emissions Factor (kgCO2vkm-1)':'mean'})
avg_emp_join_bike_emi_df = emp_join_bike_emi_df.groupby(by=['Vehicle Category'], as_index=False).agg({'Emissions Factor (kgCO2vkm-1)':'mean'})
avg_emp_join_bus_taxi_df = emp_join_bus_taxi_df.groupby(by=['Vehicle Category'], as_index=False).agg({'Emissions Factor (kgCO2pkm-1)':'mean'})
avg_emp_join_rail_df = emp_join_rail_df.groupby(by=['Vehicle Category'], as_index=False).agg({'Emissions Factor (kgCO2pkm-1)':'mean'})

temp_dist_final = final_df_output.copy()

temp_dist_final.rename(columns ={'Transport Mode':'Vehicle Category'}, inplace=True)
final_df_output_mode = pd.merge(temp_dist_final, total_emp_by_each_mode, on =['Vehicle Category'], how= 'outer' )
final_df_output_avg_dis = pd.merge(final_df_output_mode, avg_distance_each_mode, on =['Vehicle Category'], how= 'outer')
final_df_output_ave_car_ef = pd.merge(final_df_output_avg_dis, avg_emp_join_car_emi_df, on =['Vehicle Category'], how= 'outer')
final_df_output_ave_bike_ef = pd.merge(final_df_output_ave_car_ef, avg_emp_join_bike_emi_df, on =['Vehicle Category'], how= 'outer')
final_df_output_ave_bus_taxi_ef = pd.merge(final_df_output_ave_bike_ef, avg_emp_join_bus_taxi_df, on =['Vehicle Category'], how= 'outer')
final_df_output_ave_rail_ef_F = pd.merge(final_df_output_ave_bus_taxi_ef, avg_emp_join_rail_df, on =['Vehicle Category'], how= 'outer')



final_df_output_ave_rail_ef_F = final_df_output_ave_rail_ef_F.fillna(0)


#Setting perticular column to be zero  # test avg method

# final_df_output_ave_rail_ef_F.loc[final_df_output_ave_rail_ef_F['Employee Id_x']==17.0, 'Carbon Emmission/emp(kgCO2)'] = 0
# final_df_output_ave_rail_ef_F.loc[final_df_output_ave_rail_ef_F['Employee Id_x']==13.0, 'Carbon Emmission/emp(kgCO2)'] = 0
# final_df_output_ave_rail_ef_F.loc[final_df_output_ave_rail_ef_F['Employee Id_x']==21.0, 'Carbon Emmission/emp(kgCO2)'] = 0
# final_df_output_ave_rail_ef_F.loc[final_df_output_ave_rail_ef_F['Employee Id_x']==12.0, 'Carbon Emmission/emp(kgCO2)'] = 0
# final_df_output_ave_rail_ef_F.loc[final_df_output_ave_rail_ef_F['Employee Id_x']==19.0, 'Carbon Emmission/emp(kgCO2)'] = 0

# Calculating avg emission for each vehicle type
final_df_output_ave_rail_ef_F['Car Ave Emi'] = final_df_output_ave_rail_ef_F.apply(lambda x : x['Emp % of each mode']*total_emp_count/100*x['Avg distance of each mode']*2*235*x['Emissions Factor (kgCO2vkm-1)_x']
                                                                                     if (x['Vehicle Category']=="Car" and (x['Carbon Emmission/emp(kgCO2)']==0 or x['Carbon Emmission/emp(kgCO2)']==""))
                                                                                     else 0, axis=1)

final_df_output_ave_rail_ef_F['Bike Ave Emi'] = final_df_output_ave_rail_ef_F.apply(lambda x : x['Emp % of each mode']*total_emp_count/100*x['Avg distance of each mode']*2*235*x['Emissions Factor (kgCO2vkm-1)_y']
                                                                                     if (x['Vehicle Category']=="Motorbike" and (x['Carbon Emmission/emp(kgCO2)']==0 or x['Carbon Emmission/emp(kgCO2)']=="" ))
                                                                                     else 0, axis=1)

final_df_output_ave_rail_ef_F['Rail Ave Emi'] = final_df_output_ave_rail_ef_F.apply(lambda x : x['Emp % of each mode']*total_emp_count/100*x['Avg distance of each mode']*2*235*x['Emissions Factor (kgCO2pkm-1)_y']
                                                                                     if ((x['Vehicle Category'].__contains__("Rail") or x['Vehicle Category'].__contains__("Subways") or x['Vehicle Category'].__contains__("Tram")) and (x['Carbon Emmission/emp(kgCO2)']==0 or x['Carbon Emmission/emp(kgCO2)']=="" ))
                                                                                     else 0, axis=1)

final_df_output_ave_rail_ef_F['Bus Ave Emi'] = final_df_output_ave_rail_ef_F.apply(lambda x : x['Emp % of each mode']*total_emp_count/100*x['Avg distance of each mode']*2*235*x['Emissions Factor (kgCO2pkm-1)_x']
                                                                                     if (x['Vehicle Category']== "Bus" and (x['Carbon Emmission/emp(kgCO2)']==0 or x['Carbon Emmission/emp(kgCO2)']==""))
                                                                                     else 0, axis=1)

final_df_output_ave_rail_ef_F['taxi Ave Emi'] = final_df_output_ave_rail_ef_F.apply(lambda x : x['Emp % of each mode']*total_emp_count/100*x['Avg distance of each mode']*2*235*x['Emissions Factor (kgCO2pkm-1)_x']
                                                                                     if (x['Vehicle Category']== "Taxi" and (x['Carbon Emmission/emp(kgCO2)']==0 or x['Carbon Emmission/emp(kgCO2)']==""))
                                                                                     else 0, axis=1)

# Combining avg emission
final_df_output_ave_rail_ef_F['Average method Carbon Emmission/emp(kgCO2)'] = final_df_output_ave_rail_ef_F['Carbon Emmission/emp(kgCO2)'] + final_df_output_ave_rail_ef_F['Bus Ave Emi'] + final_df_output_ave_rail_ef_F['Car Ave Emi'] + final_df_output_ave_rail_ef_F['Bike Ave Emi'] + final_df_output_ave_rail_ef_F['Rail Ave Emi'] + final_df_output_ave_rail_ef_F['taxi Ave Emi']

final_df_output_ave_rail_ef_F['CO2e_Calculation_Method_New'] = final_df_output_ave_rail_ef_F.apply(lambda x : "Average Method"
                                                                                     if (x['Carbon Emmission/emp(kgCO2)']==0 and x['CO2e_Calculation_Method']!="Teleworking")
                                                                                     else x['CO2e_Calculation_Method'], axis=1)


final_df_output_ave_rail_ef_F['CO2e_Calculation_Method'] = final_df_output_ave_rail_ef_F['CO2e_Calculation_Method_New']

final_df_output_ave_rail_ef_F['Carbon Emmission/emp(kgCO2)'] = final_df_output_ave_rail_ef_F['Average method Carbon Emmission/emp(kgCO2)']

final_df_output_ave_rail_ef_F['Employee Id_x'] = final_df_output_ave_rail_ef_F['Employee Id_x'].astype(int)
final_df_output_ave_rail_ef_F['Year_x'] = final_df_output_ave_rail_ef_F['Year_x'].astype(int)

final_df_output_ave_rail_ef_F['Carbon Emmission/emp(kgCO2)'] = np.round(final_df_output_ave_rail_ef_F['Carbon Emmission/emp(kgCO2)'], decimals = 3)

final_df_output_ave_rail_ef_F['EmployeeName'] = final_df_output_ave_rail_ef_F['Employee Name_x']
final_df_output_ave_rail_ef_F['EmployeeId'] = final_df_output_ave_rail_ef_F['Employee Id_x']
final_df_output_ave_rail_ef_F['OrganisationName'] = final_df_output_ave_rail_ef_F['Organization Name_x']
final_df_output_ave_rail_ef_F['WorkRegion'] = final_df_output_ave_rail_ef_F['Work Region/Country_x']
final_df_output_ave_rail_ef_F['TrasportMode'] = final_df_output_ave_rail_ef_F['Vehicle Category']
final_df_output_ave_rail_ef_F['FuelType'] = final_df_output_ave_rail_ef_F['Fuel Type_x']
final_df_output_ave_rail_ef_F['CO2eCalculationMethod'] = final_df_output_ave_rail_ef_F['CO2e_Calculation_Method']
final_df_output_ave_rail_ef_F['Scope'] = final_df_output_ave_rail_ef_F['Scope']
final_df_output_ave_rail_ef_F['Year'] = final_df_output_ave_rail_ef_F['Year_x']
final_df_output_ave_rail_ef_F['Carbon_Cooling_Emmission/emp(kgCO2)'] = final_df_output_ave_rail_ef_F['Carbon cooling Emmission/emp(kgCO2)']
final_df_output_ave_rail_ef_F['Carbon_Home_Appliances_Emmission/emp(kgCO2)'] = final_df_output_ave_rail_ef_F['Carbon home appliances Emmission/emp(kgCO2)']
final_df_output_ave_rail_ef_F['Carbon_Heating_Emmission/emp(kgCO2)'] = final_df_output_ave_rail_ef_F['Carbon heating Emmission/emp(kgCO2)']
final_df_output_ave_rail_ef_F['EmissionCO2e'] = final_df_output_ave_rail_ef_F['Carbon Emmission/emp(kgCO2)']
final_df_output_ave_rail_ef_F['Co2eUnit'] = "Kg"

output_final = ['EmployeeName', 'EmployeeId', 'OrganisationName', 'WorkRegion', 'TrasportMode', 'FuelType', 'CO2eCalculationMethod', 'Scope' ,'Year','Carbon_Cooling_Emmission/emp(kgCO2)','Carbon_Home_Appliances_Emmission/emp(kgCO2)','Carbon_Heating_Emmission/emp(kgCO2)','EmissionCO2e', 'Co2eUnit']

final_df_output_ave_rail_ef_F.to_csv('work/final_output.csv', columns=output_final, index=False)

     
     

/tmp/ipykernel_57/3678805202.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_join_tele_df.fillna(0, inplace=True)
